In [1]:
!pip install --upgrade requests googletrans together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 49.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pillow
    Found existing installation: pillow 11.0.0
    Uninstalling pillow-11.0.0:
      Successfully uninstalled pillow-11.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
plotnine 0.14.4 requires matplotlib>=3.8.0, but you have matplotlib 3.7.5 which is incompatible.


In [2]:
import os
import json
import time
import requests
import re
from googletrans import Translator 
from together import Together
from dotenv import load_dotenv

load_dotenv()
MODEL_NAME = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo"  
COINAPI_KEY = os.getenv("COINAPI_KEY") 
TOGETHER_API_KEY= os.getenv("TOGETHER_API_KEY")  
RATE_LIMIT_DELAY = 1   


In [21]:
CRYPTO_NAME_TO_SYMBOL = {
    "Bitcoin": "BTC",
    "Ethereum": "ETH",
    "Binance Coin": "BNB",
    "Ripple": "XRP",
    "Dogecoin": "DOGE",
    "Litecoin": "LTC",
    "Cardano": "ADA",
    "Polkadot": "DOT",
    "Solana": "SOL",
}

conversation_history = []
price_cache = {}
last_symbol = None
current_language = "English"


def generate_text_with_conversation(messages, model=MODEL_NAME):
    """
    Generates text using the Together AI API, with comprehensive error handling.
    """
    try:
        from together import Together

        client = Together()

        # Create a chat completion request to the model
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            max_tokens=512,
            temperature=0.7,
            top_p=0.7,
            repetition_penalty=1.1,
        )

        # Process the response to extract content
        if (
            response
            and hasattr(response, "choices")
            and isinstance(response.choices, list)
            and len(response.choices) > 0
        ):
            choice = response.choices[0]
            if choice and hasattr(choice.message, "content"):
                content = choice.message.content
                if isinstance(content, str):
                    return content.strip()
                else:
                    return handle_error("Content is not a string.")
            else:
                return handle_error("Could not extract content from response.")
        else:
            return handle_error("Unexpected response format.")

    except ImportError as ie:
        return handle_error(f"TogetherAI module import error: {ie}")
    except Exception as e:
        return handle_error(f"LLama 3 Error details: {e}")


def handle_error(message):
    """
    Handles error messages by logging them and returning a user-friendly message.
    """
    print(f"Error: {message}")
    return f"Error: {message}"


def get_crypto_price(symbol):
    """
    Fetches the current price from CoinAPI with caching and error handling.
    """
    if symbol in price_cache:
        return (
            f"The current price of {symbol.upper()} is: {price_cache[symbol]} USD (from cache)"
        )

    time.sleep(RATE_LIMIT_DELAY)  # Respect rate limits

    try:
        # Construct the API request URL
        url = f"https://rest.coinapi.io/v1/exchangerate/{symbol}/USD"
        headers = {"X-CoinAPI-Key": COINAPI_KEY}

        # Make the API request
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an error for bad responses

        data = response.json()

        # Extract the price from the response data
        if data and "rate" in data:
            price = data["rate"]
            price_cache[symbol] = price  # Cache the price for future requests
            return f"The current price of {symbol.upper()} is: {price} USD"
        else:
            return handle_error("API returned unexpected data.")

    except requests.exceptions.RequestException as e:
        return handle_error(f"Request error occurred: {e}")
    except json.JSONDecodeError as json_err:
        return handle_error(f"JSON decode error occurred: {json_err}")
    except Exception as e:
        return handle_error(f"An unexpected error occurred: {e}")


def translate_to_english(text):
    """
    Translates input text to English using LLaMA model.
    """
    messages = [
        {"role": "system", "content": "You are a translator, translate the following text to English"},
        {"role": "user", "content": text},
    ]


    translation_response = generate_text_with_conversation(messages)

    return translation_response


def process_query(user_input):
    """
    Processes user queries, maintaining conversation context and handling translations.
    """
    global conversation_history, current_language, last_symbol

    try:

        if "change language to" in user_input.lower():
            target_language = user_input.lower().split("change language to")[-1].strip()
            current_language = target_language.capitalize()
            return f"The response language has been set to {current_language}, however, all responses will remain in English."

        # Translate user input to English for processing
        user_input_en = translate_to_english(user_input)

        symbol = None

        for crypto_name in CRYPTO_NAME_TO_SYMBOL.keys():
            if crypto_name.lower() in user_input_en.lower():
                symbol = get_symbol_from_name(crypto_name)
                break

        if not symbol:
            symbol = "BTC"

        if "price" in user_input_en.lower():
            crypto_price = get_crypto_price(symbol)
            if crypto_price:
                conversation_history.append(crypto_price)
                last_symbol = symbol  # Store the symbol for follow-up questions
            return crypto_price

        if last_symbol and any(
            keyword in user_input_en.lower()
            for keyword in ["what", "how", "why", "history", "future", "details", "aspects"]
        ):
            messages = [
                {
                    "role": "system",
                    "content": (
                        "You are an expert AI assistant specialized in cryptocurrency."
                        f"Provide accurate and insightful information about {last_symbol}."
                        "Focus on answering the user's specific question with details about"
                        "its history, future prospects, and relevant aspects."
                        "You do not offer financial advice."
                    ),
                },
                {"role": "user", "content": user_input_en},
            ]

            ai_response = generate_text_with_conversation(messages)

            if ai_response:
                conversation_history.append(ai_response)

            return ai_response

        # Prepare messages for AI assistant's response generation for general queries
        messages = [
            {
                "role": "system",
                "content": (
                    "You are an expert AI assistant specialized in cryptocurrency."
                    "Your capabilities include retrieving prices, insights into historical data, "
                    "and market predictions. You do not offer financial advice."
                ),
            },
            {"role": "user", "content": user_input_en},
        ]

        # Generate AI response based on user query
        ai_response = generate_text_with_conversation(messages)

        if ai_response:
            conversation_history.append(ai_response)

        return ai_response

    except Exception as top_level_error:
        print(f"A top-level error occurred: {top_level_error}")
        return (
            "Apologies, an unexpected error occurred: "
            f"{str(top_level_error)}. Please check the console for details."
        )


def get_symbol_from_name(crypto_name):
    """
    Returns the symbol for a given cryptocurrency name.
    """
    return CRYPTO_NAME_TO_SYMBOL.get(crypto_name.title(), None)


def main():
    """
    Main function to start the Cryptocurrency Assistant.
    """
    print(
        "Starting the Cryptocurrency Assistant.\n Type 'change language to <desired language>' to change the language.\nType 'exit' to quit."
    )

    while True:
        user_query = input("Enter your query: ")

        if user_query.lower() == "exit":
            break

        response = process_query(user_query)

        print(response)


if __name__ == "__main__":
    main()


Starting the Cryptocurrency Assistant.
 Type 'change language to <desired language>' to change the language.
Type 'exit' to quit.


Enter your query:  change language to French


The response language has been set to French, however, all responses will remain in English.


Enter your query:  prix du BTC?


The current price of BTC is: 97274.2608670571 USD


...

Enter your query:  what is the price of ETH?


The current price of ETH is: 2706.41333511922 USD


Enter your query:  Discuss market trends and predictions?


It seems you've provided a general overview of market trends and predictions across various industries. However, I'll focus on the cryptocurrency aspect, specifically Ethereum (ETH).

**Ethereum Market Trends:**

1. **Decentralized Finance (DeFi) Growth**: Ethereum's DeFi ecosystem has experienced significant growth, with total value locked (TVL) reaching new heights. This trend is expected to continue, driven by innovative lending protocols, yield farming, and decentralized exchanges.
2. **Non-Fungible Tokens (NFTs)**: NFTs have gained popularity, allowing creators to monetize unique digital assets. Ethereum's NFT market is expected to expand, with increased adoption in gaming, art, and collectibles.
3. **Layer 2 Scaling Solutions**: Ethereum's scalability issues have led to the development of layer 2 scaling solutions, such as Optimism, Polygon, and Arbitrum. These platforms aim to reduce transaction costs and increase throughput, making Ethereum more viable for mainstream adoption.


Enter your query:  exit
